# Задание по программированию: Рекомендательные системы

## *Описание задачи*

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать **recall@k** и **precision@k**.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

### *Входные данные*

Вам дается две выборки с пользовательскими сессиями - **id**-шниками просмотренных и **id**-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: **id** просмотренных товаров через , затем идёт ; после чего следуют **id** купленных товаров (если такие имеются), разделённые запятой. Например, **1,2,3,4**; или **1,2,3,4;5,6**.

Гарантируется, что среди **id** купленных товаров все различные.

***Важно:***

- Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
- Если товар не встречался в обучающей выборке, его популярность равна 0.
- Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
- Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и **k** в **recall@k** / **precision@k**.

## *Задание*

1. На обучении постройте частоты появления **id** в просмотренных и в купленных (**id** может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:
    - сортировка просмотренных **id** по популярности (частота появления в просмотренных),
    - сортировка просмотренных **id** по покупаемости (частота появления в покупках).
3. Для данных алгоритмов выпишите через пробел **AverageRecall@1**, **AveragePrecision@1**, **AverageRecall@5**, **AveragePrecision@5** на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

### *Дополнительные вопросы*

1. Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров **recall@k** меняется. Подумайте, как оценить минимальное и максимальное значение **recall@k** в зависимости от правила сортировки.
2. Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

In [1]:
import pandas as pd

train = pd.read_csv('coursera_sessions_train.txt', sep=';', header=None, names = ['views', 'buys'])
X_test = pd.read_csv('coursera_sessions_test.txt', sep=';', header=None, names = ['views', 'buys'])

In [2]:
import numpy as np

def string_features_to_int(data, nan_place):
    views, buys = list(), list()
    for view, buy in zip(data['views'], data['buys']):
        views.append([int(el) for el in view.split(',')])
        if type(buy) == float:
            buys.append(nan_place)
        else:
            buys.append([int(el) for el in buy.split(',')])
    return views, buys
views, buys = string_features_to_int(train, nan_place=[-1])

In [3]:
X_train = pd.DataFrame()
views, X_buys = string_features_to_int(train, nan_place=np.nan)
X_train['views'] = views
X_train['buys']  = X_buys
X_train.dropna(inplace=True)
X_train.reset_index(drop=True, inplace=True)
X_train.head()

,views,buys
0,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]"
1,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86]
2,"[138, 198, 199, 127]",[199]
3,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303]
4,"[352, 353, 352]",[352]


In [4]:
from collections import Counter

train['views'] = views
train['buys']  = buys

views_cnt = Counter()
buys_cnt  = Counter()
for v_lst, b_lst in zip(train['views'], train['buys']):
    for v_el in v_lst:
        views_cnt[v_el] += 1
    for b_el in b_lst:
        buys_cnt[b_el] += 1

In [5]:
test_views, test_buys = string_features_to_int(X_test, nan_place=np.nan)
X_test['views'] = test_views
X_test['buys']  = test_buys
X_test.dropna(inplace=True)
X_test.reset_index(drop=True, inplace=True)
X_test.head()

,views,buys
0,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]"
1,"[158, 159, 160, 159, 161, 162]",[162]
2,"[200, 201, 202, 203, 204]","[201, 205]"
3,"[371, 372, 371]","[371, 373]"
4,[422],[422]


In [6]:
def predict(viewed, k, method='popular'):
    unique_viewed = np.array(viewed)[np.sort(np.unique(viewed, return_index=True)[1])]    
    k = min(len(viewed), k)    
    ratings = []
    
    for item in unique_viewed:
        if method == 'popular':
            ratings.append(-views_cnt[item])
        else:
            ratings.append(-buys_cnt[item])
    
    sorted_items = np.argsort(ratings, kind='mergesort')
    return list(unique_viewed[sorted_items])[0:k]

In [7]:
def recall(predicted, buyed):
    rb = [x for x in buyed if x in predicted]
    return len(rb)/float(len(buyed))

In [8]:
def precision(predicted, buyed, k):
    rb = [x for x in buyed if x in predicted]
    return len(rb)/float(k)

In [9]:
def get_stats(data, method='popular'):
    ar1 = np.mean([recall   (predict(v, 1, method=method), b)    for v, b in zip(data['views'], data['buys'])])
    ap1 = np.mean([precision(predict(v, 1, method=method), b, 1) for v, b in zip(data['views'], data['buys'])])
    ar5 = np.mean([recall   (predict(v, 5, method=method), b)    for v, b in zip(data['views'], data['buys'])])
    ap5 = np.mean([precision(predict(v, 1, method=method), b, 5) for v, b in zip(data['views'], data['buys'])])
    return ar1, ap1, ar5, ap5

train_popular = get_stats(X_train, 'popular')
test_popular  = get_stats(X_test, 'popular')
train_purch   = get_stats(X_train, 'purch')
test_purch    = get_stats(X_test, 'purch')
print train_popular
print test_popular
print train_purch
print test_purch

(0.44263431659495955, 0.51219512195121952, 0.82469182471261182, 0.10243902439024391)
(0.41733266203252556, 0.48130968622100956, 0.80003406635385776, 0.096261937244201901)
(0.68844949242676512, 0.80376940133037689, 0.92630730242287906, 0.1607538802660754)
(0.46062016666602978, 0.52769440654843114, 0.82018743374901959, 0.10553888130968621)


In [12]:
def write_answer(assignment_N, answer):
    with open("rank_{}.txt".format(assignment_N), "w") as fout:
        fout.write(' '.join([str(round(num, 2)) for num in answer]))

write_answer(1, train_popular)
write_answer(2, test_popular)
write_answer(3, train_purch)
write_answer(4, test_purch)